In [25]:
# Import and init h2o lib. Running on java
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
h2o.init()

col_train1 = {
 'chain_name': 'enum',
 'mall_name': 'enum',
 'revenue': 'numeric',
 #'lv1': 'enum',
 'lv2': 'enum',
 'lv3': 'enum'
 #'total_nbr_people': 'int',
 #'nbr_people_per_store_in_grunnkrets': 'numeric',
 #'counts_gr_lv2': 'numeric',
 #'counts_municipality_lv2': 'numeric',
 #'busstops_within_50m': 'numeric',
 #'busstops_within_100m': 'numeric',
 #'busstops_within_400m': 'numeric',
 #'busstops_within_800m': 'numeric',
 #'busstops_within_1500m': 'numeric',
 #'num_stores_within_100m_and_same_lvl2': 'numeric',
 #'num_stores_within_500m_and_same_lvl2': 'numeric',
 #'num_stores_within_1km_and_same_lvl2': 'numeric',
 #'num_stores_within_5km_and_same_lvl2': 'numeric',
 #'num_stores_within_10km_and_same_lvl2': 'numeric',
 #'num_stores_within_20km_and_same_lvl2': 'numeric',
 #'closest_competitor_lv2': 'numeric',
 #'mean_revenue_lv3' : 'numeric',
 #'mean_revenue_lv2' : 'numeric',
 #'median_revenue_lv3' : 'numeric',
 #'median_revenue_lv2' : 'numeric',
 #'mean_revenue_chain' : 'numeric',
 #'median_revenue_lv4' : 'numeric',
 #'mean_revenue_lv4' : 'numeric'
 #'num_stores_within_100m_and_same_lvl3': 'numeric',
 #'num_stores_within_500m_and_same_lvl3': 'numeric',
 #'num_stores_within_1km_and_same_lvl3': 'numeric',
 #'num_stores_within_5km_and_same_lvl3': 'numeric',
 #'num_stores_within_10km_and_same_lvl3': 'numeric',
 #'num_stores_within_20km_and_same_lvl3': 'numeric',
 #'closest_competitor_lv3': 'numeric',
 #'municipality_size_group': 'enum',
 #'mean_revenue_for_municipality_size_group': 'numeric',
 #'median_revenue_for_municipality_size_group': 'numeric',
 #'st_dev_of_revenue_for_municipality_size_group': 'numeric',
 #'municipality_rev_group_lv2': 'numeric',
 #'mean_revenue_for_municipality_rev_group_lv2': 'numeric'
 }


Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 hours 20 mins
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_paaske_tradku
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,352.2 Mb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [26]:
# Reading csv to h2o dataframe 

train_data = pd.read_csv('feature_data/training_set_dropped.csv')
test_data = pd.read_csv('feature_data/testing_set_dropped.csv')

# Save store ids, then remove this column
store_ids = test_data['store_id'].to_numpy()
ids = pd.DataFrame(store_ids)
ids.rename(columns={0 :'id'}, inplace=True)

train_data = train_data.drop('store_id', axis=1)
test_data = test_data.drop('store_id', axis=1)

train1 = h2o.H2OFrame(train_data, column_types=col_train1)

del col_train1['revenue']

test1 = h2o.H2OFrame(test_data, column_types=col_train1)

train1.types


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


{'chain_name': 'enum',
 'mall_name': 'enum',
 'revenue': 'real',
 'lv2': 'enum',
 'lv3': 'enum',
 'lv4': 'enum',
 'municipality_name': 'enum',
 'total_nbr_people': 'real',
 'nbr_people_per_store_in_grunnkrets': 'real',
 'counts_gr_lv2': 'real',
 'counts_municipality_lv2': 'real',
 'municipality_size_group': 'enum',
 'busstops_within_3000m': 'real',
 'busstops_within_6000m': 'real',
 'num_stores_within_100m_and_same_lvl2': 'real',
 'num_stores_within_500m_and_same_lvl2': 'real',
 'num_stores_within_1km_and_same_lvl2': 'real',
 'num_stores_within_5km_and_same_lvl2': 'real',
 'num_stores_within_10km_and_same_lvl2': 'real',
 'num_stores_within_20km_and_same_lvl2': 'real',
 'closest_competitor_lv2': 'real',
 'num_stores_within_100m_and_same_lvl3': 'real',
 'num_stores_within_500m_and_same_lvl3': 'real',
 'num_stores_within_1km_and_same_lvl3': 'real',
 'num_stores_within_5km_and_same_lvl3': 'real',
 'num_stores_within_10km_and_same_lvl3': 'real',
 'num_stores_within_20km_and_same_lvl3': 'rea

In [27]:
# Removing labels from train set
y = "revenue"
x1 = train1.columns

x1.remove(y)


In [28]:
# Running fit/training on train set with cross validation n=5
aml1 = H2OAutoML(max_models = 5, seed = 2, stopping_metric = "RMSE", include_algos=["XGBoost"])
aml1.train(x = x1, y = y, training_frame = train1)
print("1")
aml2 = H2OAutoML(max_models = 5, seed = 2, stopping_metric = "RMSE", include_algos=["DRF"])
aml2.train(x = x1, y = y, training_frame = train1)
print("2")
aml3 = H2OAutoML(max_models = 5, seed = 2, stopping_metric = "RMSE", include_algos=["GBM"])
aml3.train(x = x1, y = y, training_frame = train1)
print("3")
aml4 = H2OAutoML(max_models = 5, seed = 2, stopping_metric = "RMSE", include_algos=["GLM"])
aml4.train(x = x1, y = y, training_frame = train1)
print("4")
aml5 = H2OAutoML(max_models = 5, seed = 2, stopping_metric = "RMSE", include_algos=["DeepLearning"])
aml5.train(x = x1, y = y, training_frame = train1)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
1
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
2
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
3
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
4
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,0.5284466,0.0103034,0.5292163,0.5242419,0.5402945,0.5135021,0.534978
mean_residual_deviance,0.4720985,0.0138381,0.4671244,0.4575998,0.4839055,0.4624734,0.4893894
mse,0.4720985,0.0138381,0.4671244,0.4575998,0.4839055,0.4624734,0.4893894
null_deviance,2509.266,89.33852,2573.5747,2597.0535,2543.5024,2443.9497,2388.25
r2,0.5301257,0.0172740,0.53557,0.5572926,0.5165689,0.5261011,0.5150961
residual_deviance,1177.8005,33.818947,1194.437,1149.0332,1229.6039,1158.0333,1157.8954
rmse,0.6870354,0.0100521,0.683465,0.6764613,0.6956332,0.6800540,0.6995637
rmsle,0.2875077,0.0054191,0.2885019,0.2816874,0.2892418,0.2829377,0.2951700


In [29]:
lb1 = aml1.leaderboard
lb2 = aml2.leaderboard
lb3 = aml3.leaderboard
lb4 = aml4.leaderboard
lb5 = aml5.leaderboard

In [30]:
# Showing best perforimg models
lb1.head()
lb2.head()
lb3.head()
lb4.head()
lb5.head()

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_27_20221112_143019,0.68692,0.47186,0.528233,0.287427,0.47186
GBM_5_AutoML_27_20221112_143019,0.689075,0.474825,0.529214,0.287317,0.474825
GBM_1_AutoML_27_20221112_143019,0.691221,0.477787,0.530724,0.289131,0.477787
GBM_2_AutoML_27_20221112_143019,0.695012,0.483042,0.532766,0.289602,0.483042
GBM_4_AutoML_27_20221112_143019,0.696936,0.48572,0.533253,0.290954,0.48572
GBM_3_AutoML_27_20221112_143019,0.698676,0.488148,0.536386,0.291256,0.488148


In [31]:
preds1 = aml1.predict(test1)
preds2 = aml2.predict(test1)
preds3 = aml3.predict(test1)
preds4 = aml4.predict(test1)
preds5 = aml5.predict(test1)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
drf prediction progress: |

/home/paaske/.local/lib/python3.8/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'chain_name' has levels not trained on: ["24 7 TRENINGSSENTER", "ACE SHOPS", "AVANCIA SPORT", "BADEMILJØ", "BADERINGEN", "BAROKK FRISØR", "BERTONI NORWAY", "BIRK SPORT", "BLUE ENERGY", "DESINESS", ...26 not listed..., "SKOGSTAD DETALJ", "SKY FITNESS", "TIGER OF SWEDEN", "TIPPY", "VARMEFAG INVEST", "VITA EXCLUSIVE", "YOGIS", "ZAVANNA", "ZAXIZ FRISØR", "nan"]
  warnings.warn(w)
/home/paaske/.local/lib/python3.8/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'mall_name' has levels not trained on: ["Aamodt Senter", "Bardufoss Torgsenter", "Bellevuesenteret", "Borgeåsen Senter", "CC Mart'n", "Combisenteret Fiskå", "Coop Mega Hønefoss", "Coop Mega Øvre Årdal", "Cuben Kjøpesenter", "De Syv Søstre kjøpesenter", ...40 not listed..., "Sundt Motehus", "Svelvik Nærsenter", "Tebo Senter", "Trekanten Senter Kristiansand", "Tårnhuset", "Varehuset Rosendal", "Vik Torg", 

███████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [32]:
#print variable importance
#corr = aml1.varimp(use_pandas = True)
#corr['sum'] = corr.sum(axis=1)
#corr = corr.sort_values(by=['sum'])
#corr.head(30)

In [33]:
#preds = h2o.as_list(preds)
#print(preds)

In [34]:
# Convert result to desired format

import pandas as pd
import numpy as np

weights = [0.4, 0.55, 0.45, 0.6, 0.5]
sum_weights = 5
weights = pd.DataFrame(weights)


pandas_preds1 = preds1.as_data_frame()
pandas_preds1.rename(columns={'predict' :'1'}, inplace=True)
pandas_preds2 = preds2.as_data_frame()
pandas_preds2.rename(columns={'predict' :'2'}, inplace=True)
pandas_preds3 = preds3.as_data_frame()
pandas_preds3.rename(columns={'predict' :'3'}, inplace=True)
pandas_preds4 = preds4.as_data_frame()
pandas_preds4.rename(columns={'predict' :'4'}, inplace=True)
pandas_preds5 = preds5.as_data_frame()
pandas_preds5.rename(columns={'predict' :'5'}, inplace=True)

pandas_preds = pandas_preds1.join(pandas_preds2).join(pandas_preds3).join(pandas_preds4).join(pandas_preds5)

pandas_preds['predicted'] = pandas_preds.sum(axis=1)/sum_weights


#pandas_preds.rename(columns={'predict' :'predicted'}, inplace=True)
pandas_preds['predicted'] = np.expm1(pandas_preds['predicted'])

def save_submission(pred):
    predicted = pd.DataFrame(pred, columns = ['predicted'])
    output = pd.concat([ids,predicted],axis=1)
    output.drop(output.tail(1).index,inplace=True)

    output.to_csv('h2o.csv', index=False)
    
save_submission(pandas_preds)




In [36]:
pandas_preds
#aml.explain(frame = train_data)

,1,2,3,4,5,predicted
0,1.588889,1.544443,1.545972,1.654044,1.545972,3.834916
1,1.927781,1.552513,1.305719,2.088979,1.305719,4.135320
2,3.427395,3.421790,3.265458,3.309794,3.265458,27.162154
3,1.848284,1.657216,1.781260,1.951046,1.781260,5.072760
4,1.813045,1.628722,1.868888,1.781285,1.868888,5.002438
...,...,...,...,...,...,...
8573,0.910785,1.036550,1.092826,1.013022,1.092826,1.798831
8574,0.677589,0.780721,0.658567,0.689944,0.658567,0.999861
8575,0.596473,0.581657,0.759152,0.565934,0.759152,0.920285
8576,0.613686,0.612754,0.766875,0.537539,0.766875,0.933914
